# Use Case - UR5e
A use case representing a UR5e robot using a digital twin.

This notebook controls the physical counterpart accessing the ROS driver to forward operation invocations via the RTDE control scripts

In [1]:
# Setup paths

import os
import sys

SCRIPT_PATH = os.path.relpath("../Scripts/")
GRPC_GEN_PATH = os.path.relpath("../FSR.DigitalTwin.Generated/GRPC.Generated/")
PROTOS_GEN_PATH = os.path.relpath("../FSR.DigitalTwin.Generated/Message.Generated/")

cwd = os.path.abspath(os.getcwd())

sys.path.append(os.path.join(cwd, SCRIPT_PATH))
sys.path.append(os.path.join(cwd, GRPC_GEN_PATH))
sys.path.append(os.path.join(cwd, PROTOS_GEN_PATH))

# Import Protobuf

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
from google.protobuf import message
import grpc

Establish a connection to the AAS server containing the digital representation of the assets

In [2]:
from fsr.digital_twin.workspace.digital_workspace import DigitalWorkspace

workspace = DigitalWorkspace()
workspace.connect()

Abort connection to digital twin server

In [54]:
workspace.disconnect()

Create instance of UR5e robot

In [3]:
from fsr.control.rtde.ur.UR5e import UR5e


ROBOT_HOST = "192.168.0.100"
ROBOT_PORT = 30004

robot: UR5e = UR5e(ROBOT_HOST, True)

RTDEIOInterface: realtime kernel not found, consider using a realtime kernel for better performance
Connected successfully to: 192.168.0.100 at 30004


Send robot orientation to digital workspace

In [3]:
from Protos.AasModels.V3_pb2 import KeyDTO, SubmodelElementCollectionPayloadDTO, SubmodelElementDTO
from Protos.Services.SubmodelService_pb2 import PutSubmodelElementByPathRpcRequest

SUBMODEL_ID: str = "aHR0cHM6Ly93d3cuaHMtZW1kZW4tbGVlci5kZS9pZHMvc20vNjQ5NF8yMTYyXzUwMzJfMjgxMw"

orientation = tuple([311.05599480399627, 236.4632017650581, 300.6684484538495, 278.50817289930995, 87.61544003125414, 56.53324694885834])
orientation = tuple([311.05599480399627, 236.4632017650581, -60.6684484538495, 278.50817289930995, 87.61544003125414, 56.53324694885834])
# orientation = robot.get_joint_positions_deg()

request = PutSubmodelElementByPathRpcRequest()
request.submodelId = SUBMODEL_ID
request.submodelElement
request.submodelElement.idShort = "orientation_parameters"
request.submodelElement.submodelElementType = 16

for i in range(len(orientation)):
    jointProp = SubmodelElementDTO()
    jointProp.idShort = "joint_" + str(i) + "_z"
    jointProp.submodelElementType = 10
    jointProp.property.valueType = 9
    jointProp.property.value = str(orientation[i])
    request.submodelElement.submodelElementCollection.value.append(jointProp)

orientation_key = KeyDTO()
orientation_key.type = 22
orientation_key.Value = "orientation_parameters"
request.path.append(orientation_key)
response = workspace.api_bridge.aas_api.submodel.PutSubmodelElementByPath(request)
if response.statusCode >= 400:
    raise RuntimeError("Failed to push data!")

orientation

(311.05599480399627,
 236.4632017650581,
 -60.6684484538495,
 278.50817289930995,
 87.61544003125414,
 56.53324694885834)